In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

NameError: name 'page' is not defined

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [31]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  This
  <br/>
  Afternoon
 </p>
 <p>
  <img alt="This Afternoon: Sunny, with a high near 54. Northeast wind around 7 mph. " class="forecast-icon" src="newimages/medium/few.png" title="This Afternoon: Sunny, with a high near 54. Northeast wind around 7 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 54 °F
 </p>
</div>


In [32]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

ThisAfternoon
Sunny
High: 54 °F


In [33]:
img = tonight.find("img")
desc = img['title']
print(desc)

This Afternoon: Sunny, with a high near 54. Northeast wind around 7 mph. 


In [34]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday']

In [35]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Sunny', 'Partly Cloudythen ChanceRain', 'Rain', 'Rain', 'Rain', 'Rain Likely', 'Rain', 'Rain Likely', 'Slight ChanceRain thenMostly Sunny']
['High: 54 °F', 'Low: 42 °F', 'High: 55 °F', 'Low: 47 °F', 'High: 60 °F', 'Low: 49 °F', 'High: 59 °F', 'Low: 45 °F', 'High: 56 °F']
['This Afternoon: Sunny, with a high near 54. Northeast wind around 7 mph. ', 'Tonight: A 30 percent chance of rain after 4am.  Partly cloudy, with a low around 42. East wind 6 to 9 mph. ', 'Saturday: Rain, mainly after 10am.  High near 55. North wind 6 to 8 mph becoming south in the afternoon.  Chance of precipitation is 80%. New precipitation amounts between a tenth and quarter of an inch possible. ', 'Saturday Night: Rain.  Low around 47. South wind 6 to 13 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. ', 'Sunday: Rain, mainly before 4pm.  High near 60. Southwest wind 8 to 14 mph.  Chance of precipitation is 80%. New precipitation amounts between a

In [49]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,ThisAfternoon,Sunny,High: 54 °F,"This Afternoon: Sunny, with a high near 54. No..."
1,Tonight,Partly Cloudythen ChanceRain,Low: 42 °F,Tonight: A 30 percent chance of rain after 4am...
2,Saturday,Rain,High: 55 °F,"Saturday: Rain, mainly after 10am. High near ..."
3,SaturdayNight,Rain,Low: 47 °F,Saturday Night: Rain. Low around 47. South wi...
4,Sunday,Rain,High: 60 °F,"Sunday: Rain, mainly before 4pm. High near 60..."
5,SundayNight,Rain Likely,Low: 49 °F,"Sunday Night: Rain likely, mainly after 10pm. ..."
6,Monday,Rain,High: 59 °F,Monday: Rain. High near 59. Chance of precipi...
7,MondayNight,Rain Likely,Low: 45 °F,"Monday Night: Rain likely, mainly before 10pm...."
8,Tuesday,Slight ChanceRain thenMostly Sunny,High: 56 °F,Tuesday: A slight chance of rain before 10am. ...


In [73]:
temp_nums = weather["temp"].str.extract('(\d+)', expand=True)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

,0
0,54
1,42
2,55
3,47
4,60
5,49
6,59
7,45
8,56


In [74]:
weather["temp_num"].mean()

51.888888888888886

In [75]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [54]:
weather[is_night]

,period,short_desc,temp,desc,is_night
1,Tonight,Partly Cloudythen ChanceRain,Low: 42 °F,Tonight: A 30 percent chance of rain after 4am...,True
3,SaturdayNight,Rain,Low: 47 °F,Saturday Night: Rain. Low around 47. South wi...,True
5,SundayNight,Rain Likely,Low: 49 °F,"Sunday Night: Rain likely, mainly after 10pm. ...",True
7,MondayNight,Rain Likely,Low: 45 °F,"Monday Night: Rain likely, mainly before 10pm....",True
